# Home work "CTC-loss"

Задача: реализовать прямой (forward) проход и вчислить матрицу $\alpha_t(s)$. Значения этой матрицы должны равняться
- $\alpha_t(s)$ в случае если валидные пути проходят через данную ячейку
- 0.0 в противном случае

Размерности входов и результата см. по коду.

Ноутбук с решением присыслать на voropaev@corp.mail.ru Тему письма пишите пожалуйста в формате "[МИФИ] Фамилия"

Deadline: 21.12.2018

In [1]:
import torch

In [2]:
# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,1,4,5,4,3,2,1,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1,  0,  0., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0,  0., 1., 0., 0., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],], dtype=torch.float32)
print(y.shape)

torch.Size([14, 6])


In [3]:
def compute_ctc_alpha(l, y):
    """
    Функция, вычисляющая матрицу $\alpha$ для данного входа.
    @param l метки символов строки, размерностью [L,]
    @param y предсказанные сетью вероятности для каждого фрейма. Размерность [T, Lexicon+1]
    @return матрицу $\alpha$ размерностью [2*L+1, T]
    """
    
    T = y.size()[0]
    L = l.size()[0]
    
    alpha_m = torch.zeros((2*l.size()[0]+1, y.size()[0]))
    alpha_m[0][0] = y[0][0]
    alpha_m[1][0] = y[0][l[1]]
    
    for t in range(1, T):
        for s in range(2 * L + 1):
            if s % 2 == 0 :
                alpha_m[s][t] = y[t][0] * (alpha_m[s - 1][t - 1] + alpha_m[s][t - 1])
            
            else :
                cur_c = l[s // 2]
                prev_c = l[s // 2 - 1]
                
                if cur_c == prev_c :
                    alpha_m[s][t] = y[t][cur_c] * (alpha_m[s - 1][t - 1] + alpha_m[s][t - 1])
                
                else:
                    alpha_m[s][t] = y[t][cur_c] * alpha_m[s][t - 1]
                    
                    if s >= 1:
                        alpha_m[s][t] += y[t][cur_c] * alpha_m[s - 1][t - 1]
                        if s >= 2:
                            alpha_m[s][t] += y[t][cur_c] * alpha_m[s - 2][t - 1]
                        
    return alpha_m

In [4]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
assert test(), "Test faled"
    